# Download the data
 We start by Downloading the dataset

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !cp "/content/goodreads_reviews_spoiler.json.gz" -r "drive/MyDrive"

In [ ]:
# %%shell
# pip install -U --no-cache-dir gdown --pre
# gdown https://drive.google.com/uc?id=196W2kDoZXRPjzbTjM6uvTidn6aTpsFnS

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-3.13.0-cp37-none-any.whl size=9046 sha256=d0a03f2d75db1be5f9ac5432a04db60b6db4635302731364398ccc49fb234ece
  Stored in directory: /tmp/pip-ephem-wheel-cache-4n27rqbb/wheels/ba/fa/c5/12813d7496f34652c43a471e11a780e769889d06e34735c32e
Successfully built gdown
  Found existing installation: gdown 3.6.4
    Uninstalling gdown-3.6.4:
      Successfully uninstalled gdown-3.6.4
Downloading...
From: https://drive.google.com/uc?id=196W2kDoZXRPjzbTjM6uvTidn6aTpsFnS
To: /content/goodreads_reviews_spoiler.json.gz
620MB [00:05, 121MB/s]


In [10]:
pip install scikit-surprise

     |████████████████████████████████| 11.8MB 348kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617596 sha256=a1112ce368c39bc6640cb9405467fe8e55b175403d4313d12d07b6d4f07ae24c
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [1]:
import json 
import pandas as pd 
import gzip

## Now we load the dataset

In [2]:
user_ids = []
book_ids = []
rating = []
chunks = pd.read_json('/content/drive/MyDrive/goodreads_reviews_spoiler.json.gz', lines=True, chunksize = 100)
for chunk in chunks:
    user_ids.extend(chunk.iloc[:,0])
    rating.extend(chunk.iloc[:,3])
    book_ids.extend(chunk.iloc[:,5])


In [3]:
df = pd.DataFrame(list(zip(user_ids,book_ids,rating)), columns=['user_id','book_id','rating'])



We free the RAM by cleaning unneeded objects

In [4]:
chunks = None
user_ids = None
book_ids = None
rating = None

## Collaborative Filtering
In our first test we will use collaborative filtering to get the similarity matrix between the books

In [5]:
testdf=df[['user_id','rating']].groupby(df['book_id'])
listOfDictonaries=[]
indexMap = {}
reverseIndexMap = {}
ptr=0;
# go through the data and store in dictionary
for groupKey in testdf.groups.keys():
    tempDict={}
    groupDF = testdf.get_group(groupKey)
    for i in range(0,len(groupDF)):
        tempDict[groupDF.iloc[i,0]]=groupDF.iloc[i,1]
    indexMap[ptr]=groupKey
    reverseIndexMap[groupKey] = ptr
    ptr=ptr+1
    listOfDictonaries.append(tempDict)

In [6]:
from sklearn.feature_extraction import DictVectorizer

dictVectorizer = DictVectorizer(sparse=True)
vector = dictVectorizer.fit_transform(listOfDictonaries)

### Compute similarity
Here we compute the similarity between books

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
pairwiseSimilarity = cosine_similarity(vector)

In [8]:
import numpy as np

def getTopRecommandations(bookID):
    row = reverseIndexMap[bookID]
    print("------INPUT BOOK--------")
    print("Book-ID:",bookID)
    # printBookDetails(bookID)
    print("-------RECOMMENDATIONS----------")
    similarBookIDs = [print("Book-ID:",indexMap[i]) for i in np.argsort(pairwiseSimilarity[row])[-7:-2][::-1]]

### Predict
The recommendation here takes a Book and gives similar books

In [9]:
getTopRecommandations(16981)

------INPUT BOOK--------
Book-ID: 16981
-------RECOMMENDATIONS----------
Book-ID: 59950
Book-ID: 53639
Book-ID: 48467
Book-ID: 26530351
Book-ID: 12953517


In [10]:
dictVectorizer = None
vector = None
pairwiseSimilarity = None

## Surprise Library
In the second test we will try the Surpise library to get our recommendatios

In [11]:
# the data is too big for the library so 
# we have to take some of the data and test on it
df2 = df.head(10000)

In [12]:
from surprise import Dataset
from surprise import Reader

import heapq

# define the values of rating for the Surprise reader
reader = Reader(rating_scale=(0, 5))

# Loads Pandas dataframe
data = Dataset.load_from_df(df2[["user_id", "book_id", "rating"]], reader)

In [13]:
# prepare the training set
trainset = data.build_full_trainset()

### KNN Modul
Trying the Knn modul for out first test and we compute the similarity matrix

In [14]:
from surprise import KNNBasic
similarity_matrix_KNN = KNNBasic(sim_options={
        'name': 'cosine',
        'user_based': False
        })\
        .fit(trainset)\
        .compute_similarities()

Computing the cosine similarity matrix...


/usr/local/lib/python3.7/dist-packages/surprise/prediction_algorithms/algo_base.py:249: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


### SVD Modul
Now we try the SVD and we compute the similarity matrix

In [15]:
from surprise import SVD

similarity_matrix_SVD = SVD().fit(trainset).compute_similarities()

In [16]:
# this is our test subject
test_subject = '8842281e1d1347389f2ab93d60773d4d'
# Get the top K items user rated
k = 20

In [17]:
# the surprise library makes a new ids for the users for
# memory manegment, so we have to get the inner id of our test subject
test_subject_iid = trainset.to_inner_uid(test_subject)

# Get the top K items the user rated
test_subject_ratings = trainset.ur[test_subject_iid]
k_neighbors = heapq.nlargest(k, test_subject_ratings, key=lambda t: t[1])

### KNN Recommendation
here we pick the candidates from our data that are most similer to the user choices

In [18]:
from collections import defaultdict
# Default dict is basically a standard dictionary,
# the difference beeing that it doesn't throw an error
# when trying to access a key which does not exist,
# instead a new entry, with that key, is created.
candidates = defaultdict(float)

for itemID, rating in k_neighbors:
    try:
      similaritities = similarity_matrix_KNN[itemID]
      for innerID, score in enumerate(similaritities):
          candidates[innerID] += score * (rating / 5.0)
    except:
      continue

we pick the best recommendation to our user

In [19]:
from operator import itemgetter
read = {}
for itemID, rating in trainset.ur[test_subject_iid]:
  read[itemID] = 1

# Add items to list of user's recommendations
# If they are similar to their favorite movies,
# AND have not already been read.
recommendations = []

position = 0
for itemID, rating_sum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
  if not itemID in read:
    recommendations.append(trainset.to_raw_iid(itemID))
    position += 1
    if (position > 10): break # We only want top 10

for rec in recommendations:
  print("BookID: ", rec)

BookID:  4667024
BookID:  15724396
BookID:  7937843
BookID:  26114135
BookID:  28016509
BookID:  31450633
BookID:  18812405
BookID:  22839846
BookID:  22609310
BookID:  35244824
BookID:  31393997


### SVD Recommendation
now we do the same thing but with the SVD modul similarity matrix

In [20]:
candidates = defaultdict(float)

for itemID, rating in k_neighbors:
    try:
      similaritities = similarity_matrix_SVD[itemID]
      for innerID, score in enumerate(similaritities):
          candidates[innerID] += score * (rating / 5.0)
    except:
      continue

In [21]:
read = {}
for itemID, rating in trainset.ur[test_subject_iid]:
  read[itemID] = 1

recommendations = []

position = 0
for itemID, rating_sum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
  if not itemID in read:
    recommendations.append(trainset.to_raw_iid(itemID))
    position += 1
    if (position > 10): break

for rec in recommendations:
  print("BookID: ", rec)

BookID:  35275234
BookID:  34705260
BookID:  34787781
BookID:  33807229
BookID:  32054095
BookID:  36065344
BookID:  35244824
BookID:  34776338
BookID:  35604204
BookID:  24044596
BookID:  31393997


We see that each modul did a diffirent prediction.
And a small note: the SVD model can easily handle all the 1M set of data, but the KNN modul is the one with the RAM problem, but we used the same data on both for comparison purposes.